In [1]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import word_tokenize
import re
import matplotlib.pyplot as plt
import math
from nltk.corpus import stopwords


In [2]:
Data = pd.read_csv("JobDataTranslated.csv")

In [3]:
# function to preprocess speech
#Data['Job Description'].apply(lambda x: x.lower())

def clean(text):
    
    # removing paragraph numbers
    text = re.sub('[0-9]+.\t','',str(text))
    # removing new line characters
    text = re.sub('\n ','',str(text))
    text = re.sub('\n',' ',str(text))
    # removing apostrophes
    text = re.sub("'s",'',str(text))
    # removing hyphens
    text = re.sub("-",' ',str(text))
    text = re.sub("— ",'',str(text))
    # removing quotation marks
    text = re.sub('\"','',str(text))
    # removing salutations
    text = re.sub("Mr\.",'Mr',str(text))
    text = re.sub("Mrs\.",'Mrs',str(text))
    # removing any reference to outside text
    text = re.sub("[\(\[].*?[\)\]]", "", str(text))
    
    return text

# preprocessing speeches
Data['Cleaned Job Description'] = Data['Job Description'].apply(clean)

from string import punctuation

def remove_punctuation(s):
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')
    return s

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].map(remove_punctuation)

Data['Cleaned Job Description'] = Data['Cleaned Job Description'].apply(lambda x: x.lower())


In [4]:
#Data['Job Description'].apply(lambda x: x.lower())

#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
corpus = " ".join(Data['Cleaned Job Description'].tolist())
tokens = word_tokenize(corpus)
# Get the parts of speech tag for all words
answer = nltk.pos_tag(tokens)
answer_pos = [a[1] for a in answer]

# print a value count for the parts of speech
all_pos = pd.Series(answer_pos)
all_pos.value_counts().head()


NN     106773
IN      57169
JJ      45685
DT      40667
NNS     32368
dtype: int64

In [5]:
#nltk.download('stopwords')
en_stopwords = stopwords.words('english')
def remove_stopwords(s):
    global en_stopwords
    s = word_tokenize(s)
    s = " ".join([w for w in s if w not in en_stopwords])
    return s

# Create a new column of descriptions with no stopwords
Data['Cleaned Job Description NoStop'] = Data['Cleaned Job Description'].map(remove_stopwords)

# make a corpus of all the words in the job description
corpus = " ".join(Data['Cleaned Job Description NoStop'].tolist())

# This is the NLTK function that breaks a string down to its tokens
tokens = word_tokenize(corpus)

answer = nltk.pos_tag(tokens)
answer_pos = [a[1] for a in answer]

all_pos = pd.Series(answer_pos)
all_pos.value_counts().head()

NN     103824
JJ      48363
NNS     31957
VBP     14378
VBG     13640
dtype: int64

In [6]:
# import the necessary functions from the nltk library
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()

# prepare corpus from the descriptions that dont have stopwords
corpus = " ".join(Data['Cleaned Job Description NoStop'].tolist())

#tokenize words
tokenized_corpus = nltk.word_tokenize(corpus)

# lemmatize these tokens
lemmatized_tokens = [lmtzr.lemmatize(token) for token in tokenized_corpus]

# word frequencies for the lemmatized tokens
fd = nltk.FreqDist(lemmatized_tokens)

# get the top words
top_words = []
for key, value in fd.items():
    top_words.append((key, value))

# sort the list by the top frequencies
top_words = sorted(top_words, key = lambda x:x[1], reverse = True)

# keep top 10 words only



# From here fun begins

In [7]:

data = pd.read_csv(r"C:\Users\Dima\Downloads\data_merged.csv")

#data=data[data["Is requirment"]==1]
indicies = [i for i in range(len(data)) if data["Is requirment"][i]==1]

words = [data['word'][i] for i in indicies]


In [8]:
def find_word(text, word):
    
    indices = [i for i in range(len(text)) if text[i] == word]
    return indices

def find_words(text,words):
    z=[]
    for i in words:
        if (len(find_word(text,i))!=0):
            z.append(find_word(text,i))
    unlist=[z[i][j] for i in range(len(z)) for j in range(len(z[i]))]
    return unlist

useless=["you","your","and","of","for","are","a","to","the","an","is","also","or","per","in","we","with","our","us","this","on",
        "×","as","who","on"]
# loc - set of indicies with all words in text 
def get_neigbour(text,loc, left=True):
    z=[]
    if (left):
        for i in loc:
            if (i!=0 and useless.count(text[i-1])==0):
                z.append(text[i-1]+" "+text[i])
    else:
        for i in loc:
            if (i!=len(text)-1 and useless.count(text[i+1])==0):
                z.append(text[i]+" "+text[i+1])
            
    return z



req=[]
for i in range(1000):
    txt = Data["Cleaned Job Description"][i]
    x = txt.split()
    new_w=get_neigbour(x,find_words(x,words),True)
    req.append(new_w)

def ugly_list_to_cute_df():
    req_unlist = [req[i][j] for i in range(len(req)) for j in range(len(req[i]))]
    final = list(dict.fromkeys(req_unlist))

    z=[]
    for i in final:
        z.append(req_unlist.count(i))

    datta = {'frequency':  z, 'Combo': final}
    dtf = pd.DataFrame (datta, columns = ['frequency','Combo'])
    dtf=dtf.sort_values(by=['frequency'], ascending=False)
    return dtf

df = ugly_list_to_cute_df()
#df.to_csv("left_neighbours.csv")

In [9]:
def get_neigbours(text,loc):
    z=[]
    for i in loc:
        if (i!=0 and i!=len(text)-1):
            if (useless.count(text[i-1])==0 and useless.count(text[i+1])==0):
                z.append(text[i-1]+" "+text[i]+" "+ text[i+1])
            
    return z

req=[]
for i in range(1000):
    txt = Data["Cleaned Job Description"][i]
    x = txt.split()
    new_w=get_neigbours(x,find_words(x,words))
    req.append(new_w)
    
df = ugly_list_to_cute_df()
#df.to_csv("both_neigbours.csv")


In [10]:
from nltk.corpus import wordnet


def get_synonims(word):
    synonyms = []
    for syn in wordnet.synsets(word):
            for l in syn.lemmas():
                synonyms.append(l.name())
    return set(synonyms)



words_extended = set()
for i in words:
    words_extended.add(i)
    words_extended=words_extended.union(get_synonims(i))
    
def reduce(l):
    return list(dict.fromkeys(l))

# words_short directly words mentioned,words_ long - with all synonims

In [11]:
dfL = pd.read_csv(r"C:\Users\Dima\Downloads\Left neighbours - Sheet1.csv")
dfR=pd.read_csv(r"C:\Users\Dima\Downloads\Right neihbours.csv")
dfB = pd.read_csv(r"C:\Users\Dima\Downloads\Both neighbours - Sheet1.csv")

indiciesB = [i for i in range(len(dfB)) if dfB["result"][i]==1]
three_w = [dfB['Combo'][i] for i in indiciesB]


indiciesL = [i for i in range(len(dfL)) if dfL["result"][i]==1]
two_w_l = [dfL['Combo'][i] for i in indiciesL]

indiciesR = [i for i in range(len(dfR)) if dfR["result"][i]==1]
two_w_r = [dfR['Combo'][i] for i in indiciesR]

two_w=two_w_l
two_w.extend(two_w_r)


two_w.extend(three_w)
two_w=reduce(two_w)

def words_short(x=x):
    words_short=reduce(get_neigbour(x,find_words(x,words),True))
    words_short.extend(reduce(get_neigbour(x,find_words(x,words),False)))
    words_short.extend(reduce(get_neigbours(x,find_words(x,words))))
    words_short = reduce(words_short)
    return words_short

def words_long(x=x):
    words_long = reduce(get_neigbour(x,find_words(x,words_extended),True))
    words_long.extend(reduce(get_neigbour(x,find_words(x,words_extended),False)))
    words_long.extend(reduce(get_neigbours(x,find_words(x,words_extended))))
    words_long = reduce(words_long)
    return words_long


# Super list of 2 words

In [12]:
def get_super_two_list(l = two_w_l):
    lis=[]
    for i in l:
        x=i.split()
        s=get_synonims(x[1])
        f=get_synonims(x[0])
        lis.append(i)
        for k in f:
            for j in s:
                lis.append(k+" "+j)
    return list(dict.fromkeys(lis))

def get_super_three_list(l = three_w):
    lis=[]
    for i in l:
        x=i.split()
        t = get_synonims(x[2])
        s=get_synonims(x[1])
        f=get_synonims(x[0])
        lis.append(i)
        for k in f:
            for j in s:
                for m in t:
                    lis.append(k+" "+j+" "+m)
    return list(dict.fromkeys(lis))



super_list = get_super_two_list()
super_list.extend(get_super_two_list(two_w_r))
super_list.extend(get_super_three_list())
super_list = reduce(super_list)

def remove_substrings(words = two_w):
    two=[]
    three=[]
    for i in words:
        if (len(i.split())==2):
            two.append(i)
        else:
            three.append(i)
    
    
    for i in three:
        for j in two:
            if i.rfind(j)!=-1:
                two.remove(j)
    two.extend(three)
    return two

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

# After we made use of our magic algoritm

## example 1 from training set

In [13]:
tex = Data["Cleaned Job Description"][8]
x = tex.split()
Data["Job Description"][8]

"About the position Have you obtained a teaching assistant diploma or your propaedeutic certificate from the PABO? Then accept the challenge! Do you like flexible working and would you like to work at different schools in different positions, then join our pool for the Haaglanden and Rotterdam regions. We work together with more than 450 schools where we provide teaching assistants with work. In this we make the most suitable match possible with your preferences. We'll keep in touch! I will work with you to find out where you can best use your qualities and what best suits your preferences. Whether you like to teach in the lower, middle or upper classes, we have more than enough assignments that match your preferences. After the introduction, we remain connected to each other; I follow your career as a teaching assistant and often have contact with each other to ensure that you enjoy your work. Working for Maandag® means working from flexible security. You work at one or more schools i

In [14]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['very positive', 'primary education', 'teaching assistant diploma', 'flexible teaching assistant', 'can adapt well'] 

['very positive', 'primary education', 'class management', 'can adapt well', 'teaching assistant diploma', 'flexible teaching assistant'] 

['very positive', 'primary education', 'teaching assistant diploma', 'flexible teaching assistant', 'can adapt well'] 

['very positive', 'primary education', 'class management', 'can adapt well', 'teaching assistant diploma', 'flexible teaching assistant'] 



## example 2 from training set

In [15]:
tex = Data["Cleaned Job Description"][80]
x = tex.split()
Data["Job Description"][80]

"Do you want to make an impact through personnel planning and Workforce Management? Do you enjoy advising healthcare organizations on planning and scheduling? Do you want the best of the best for your clients and their employees and do you want to develop further in an inspiring environment together with other enthusiastic colleagues? Read on and apply for the position of senior WFM healthcare consultant! This is PlanMen We are a consultancy organization specialized in personnel planning, also known as Workforce Management (WFM). More than 70 colleagues daily help organizations across the country to implement and improve the WFM process. In addition to helping our clients, we also train planners and professionals in our PlanMen Academy. We actively share knowledge by organizing various events, such as the annual PlanMen Congress. Many also know us from the PlanMen WFM Awards. An award show that puts planners and organizations in the limelight. Our head office is located in Doorn, in th

In [16]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['change management', 'analytical thinking', 'analytical skills', 'personnel planning', 'day management', 'project management', 'healthcare consultant', 'strong analytical', 'advising healthcare', 'senior consultant', 'workforce management', 'personal leadership', 'demonstrable experience', 'passionate specialists', 'planning software', 'written communication skills', 'hbo language dutch'] 

['change management', 'analytical thinking', 'analytical skills', 'personnel planning', 'day management', 'project management', 'healthcare consultant', 'planning cycle', 'strong analytical', 'advising healthcare', 'sound knowledge', 'senior consultant', 'care experience', 'workforce management', 'personal leadership', 'demonstrable experience', 'area care', 'thinking level', 'passionate specialists', 'planning software', 'written communication skills', 'hbo language dutch'] 

['change management', 'analytical thinking', 'analytical skills', 'personnel planning', 'day management', 'project manageme

## example 3 from training set

In [17]:
tex = Data["Cleaned Job Description"][800]
x = tex.split()
Data["Job Description"][800]

"Position Title Site Head - 6 months assignment  02-Dec-2020  Job ID 308193BR  Job Description The Site Head for IDB will operate to produce the starting material to be processed in Drug Product sites for the production of the finished product. They will be responsible to translate the TechOps Theragnostic and Country strategies in actionable action plans including: Preparation and execution of CAPEX; constantly improving operational efficiencies in the site; ensuring the site has adequate resources and capabilities to ensure compliance, quality, service to patients, and people development. The Site Head will develop people to create an unbossed, curious and innovative culture as a basis for sustainable performance.  Major Activities:  Strategy, Culture, and People • Together with management, develop a Site Master Plan for the site outlining a 5-year plan including forecasted volumes, expenses, and Capex supporting business needs. • Create a culture of accountability based on inspired,

In [18]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['quality service', 'be responsible', 'capex supporting', 'radioligand therapy', 'leading innovation', 'risks analysis', 'nuclear medicine aaa', 'escalate critical undesired'] 

['quality service', 'be responsible', 'capex supporting', 'management develop', 'radioligand therapy', 'leading innovation', 'risks analysis', 'nuclear medicine aaa', 'escalate critical undesired'] 

['quality service', 'be responsible', 'radioligand therapy', 'capex supporting', 'leading innovation', 'risks analysis', 'nuclear medicine aaa', 'escalate critical undesired'] 

['quality service', 'be responsible', 'management develop', 'master plan', 'radioligand therapy', 'risks analysis', 'capex supporting', 'leading innovation', 'escalate critical undesired', 'nuclear medicine aaa'] 



## example 1 new vacancy

In [19]:
f = open(r"C:\Users\Dima\Downloads\Ex1.txt", "r")
x=f.read()
print(x)

x = clean(x)
x=remove_punctuation(x)
x=x.lower()
x=x.split()


Are you "from here"? Do you speak the language of the region? Are you someone who sets goals to exceed them? Are you that commercial professional who realizes growth? Then I am looking for you as a commercial employee field service!

Toorn & Boer is going to expand: the team wants to serve its customers across the board and they need you for that. Next spring they will move to the center of Veendam and they want to build the customer portfolio there together with you. You prefer to join the team as soon as possible to get it off to a flying start.

Jeffrey Toorn and Robert Boer are friends and partners. Thanks to transparency and reliability, Toorn & Boer has quickly grown into one of the largest private advisors in the region. Clients can contact them for day-to-day banking, mortgage advice, insurance and business administration. They realize tailor-made solutions for their customers through a simple working method and clear communication. Toorn & Boer has a large (private) customer p

In [20]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['hbo level', 'work experience', 'field service', 'education hbo', 'clear communication'] 

['hbo level', 'work experience', 'field service', 'education hbo', 'clear communication'] 

['hbo level', 'work experience', 'education hbo', 'field service', 'clear communication'] 

['hbo level', 'work experience', 'education hbo', 'field service', 'clear communication'] 



## Example 2 new vacancy

In [21]:
f = open(r"C:\Users\Dima\Downloads\Ex2.txt", "r")
x=f.read()
print(x)

x = clean(x)
x=remove_punctuation(x)
x=x.lower()
x=x.split()


YIELD

Yield is the partner in interior projects. From project design to school design, lighting and audiovisual solutions. We provide expert advice, from design to realization and aftercare.

Audiovisual solutions can no longer be ignored in our interior image. That is why we offer audiovisual solutions, innovative and interactive. From touchscreens to video walls, light, sound and electrical engineering. We provide advice, install and deliver ready-to-use. Knowledge and complete service are central to us.

Times change, we grow and want to continue to develop. That is why we are looking for you!

FUNCTION

You are a multi-tasker and like variety in your work. As an audiovisual technician you have knowledge about everything to do with image and sound.

In consultation with the project manager you take care of the execution on the work floor. You understand the wishes of the client and ensure that all installations are in perfect order.

You have a good command of the Dutch language an

In [22]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['project manager', 'audiovisual technician', 'have experience', 'dutch language english'] 

['expert advice', 'project manager', 'thinking level', 'audiovisual technician', 'have experience', 'project design', 'dutch language english'] 

['project manager', 'audiovisual technician', 'have experience', 'dutch language english'] 

['expert advice', 'work floor', 'project manager', 'image sound', 'thinking level', 'audiovisual technician', 'have experience', 'project design', 'dutch language english'] 



## Example 3 new vacancy

In [23]:
f = open(r"C:\Users\Dima\Downloads\Ex3.txt", "r")
x=f.read()
print(x)

x = clean(x)
x=remove_punctuation(x)
x=x.lower()
x=x.split()

You don't have to be called Harry, but you do have to be handy

We are Patient Lifting Solutions (PLS), the name says it all: solutions to lift people. We are talking about patients in healthcare who, thanks to our smart system, can be very easily lifted out of bed and moved around the room.

We design and assemble the highest quality system of all ceiling lifts in the world. We supply this solution to healthcare providers worldwide, who then sell and install it in care homes and hospitals.

The time is now right to continue to grow and professionalize. The ISO-13485 quality system that we have achieved helps with this. And because trade shows have been canceled during this time, we want to promote our product to potential distributors through other channels. So we need a marketer. Someone who thinks outside the box when it comes to reaching new (international) markets. A "handy Harry" who, in addition to his or her marketing skills, wants to contribute ideas about good design and qual

In [24]:
# just list and just words 
print(remove_substrings(intersection(words_short(x),two_w)),'\n')

# just list and extended words 
print(remove_substrings(intersection(words_short(x),super_list)),'\n')

# super list and just words 
print(remove_substrings(intersection(words_long(x),two_w)),'\n')

# super list & extended words
print(remove_substrings(intersection(words_long(x),super_list)),'\n')


['product engineer', 'supervising post', 'product developer', 'developer marketer', 'english language', 'marketing surveillance', 'education mbo', 'language dutch', 'patient lifting solutions'] 

['product engineer', 'supervising post', 'product developer', 'developer marketer', 'english language', 'thinking level', 'marketing surveillance', 'education mbo', 'language dutch', 'patient lifting solutions'] 

['product engineer', 'supervising post', 'product developer', 'developer marketer', 'english language', 'marketing surveillance', 'education mbo', 'language dutch', 'patient lifting solutions'] 

['product engineer', 'supervising post', 'product developer', 'developer marketer', 'english language', 'thinking level', 'product design', 'marketing surveillance', 'education mbo', 'language dutch', 'patient lifting solutions'] 

